In [6]:
import sys
import os
from pathlib import Path

import pandas as pd
import numpy as np

# Ensure project root is on path
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)

raw_path = Path("../data/raw/warranty_claims_synthetic.csv")
processed_dir = Path("../data/processed")
processed_dir.mkdir(parents=True, exist_ok=True)

raw_path, processed_dir


(WindowsPath('../data/raw/warranty_claims_synthetic.csv'),
 WindowsPath('../data/processed'))

In [7]:
df_raw = pd.read_csv(raw_path)

df_raw.head()
df_raw.info()
df_raw.describe(include="all")
df_raw["Final_Claim_Decision"].value_counts()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Claim_ID                   5000 non-null   object 
 1   Claim_Date                 5000 non-null   object 
 2   Vehicle_Registration_Date  5000 non-null   object 
 3   Vehicle_Failure_Date       5000 non-null   object 
 4   Vehicle_MFD_Year           5000 non-null   int64  
 5   Mileage_km                 5000 non-null   float64
 6   Part_Group                 5000 non-null   object 
 7   Subpart_Code               5000 non-null   object 
 8   Failure_Mode               5000 non-null   object 
 9   Customer_Type              5000 non-null   object 
 10  Region                     5000 non-null   object 
 11  Labor_Cost                 5000 non-null   float64
 12  Material_Cost              5000 non-null   float64
 13  Total_Cost                 5000 non-null   float

Final_Claim_Decision
Reject     2709
Approve    2291
Name: count, dtype: int64

In [8]:
df = df_raw.copy()

# Parse dates
date_cols = ["Claim_Date", "Vehicle_Registration_Date", "Vehicle_Failure_Date"]
for c in date_cols:
    df[c] = pd.to_datetime(df[c])

# Ensure numeric types
num_cols = [
    "Vehicle_MFD_Year",
    "Mileage_km",
    "Labor_Cost",
    "Material_Cost",
    "Total_Cost",
    "Burden_Ratio",
]
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce")

# Categorical columns
cat_cols = [
    "Part_Group",
    "Subpart_Code",
    "Failure_Mode",
    "Customer_Type",
    "Region",
    "Final_Claim_Decision",
    "Final_DPR_Decision",
]
for c in cat_cols:
    df[c] = df[c].astype("category")

df.head()



,Claim_ID,Claim_Date,Vehicle_Registration_Date,Vehicle_Failure_Date,Vehicle_MFD_Year,Mileage_km,Part_Group,Subpart_Code,Failure_Mode,Customer_Type,Region,Labor_Cost,Material_Cost,Total_Cost,Burden_Ratio,Final_Claim_Decision,Final_DPR_Decision
0,CLM_000001,2022-04-08,2020-06-04,2022-03-19,2020,27512.065570,Chassis,CHS_A,No_Start,Retail,Region_2,154.603156,388.187881,542.791037,0.382099,Approve,Reject
1,CLM_000002,2024-04-27,2023-10-12,2024-04-14,2023,4840.819371,Body,BODY_B,No_Start,Retail,Region_3,142.265157,68.022091,210.287249,0.791511,Approve,Reject
2,CLM_000003,2023-12-18,2018-09-16,2023-11-17,2018,74398.919284,Electronics,ELEC_A,Leak,Retail,Region_2,159.957023,426.241409,586.198432,0.362170,Reject,Reject
3,CLM_000004,2023-04-26,2017-08-15,2023-03-17,2017,58781.129706,Body,BODY_A,Electrical_Issue,Retail,Region_3,278.097472,168.986598,447.084071,0.621667,Approve,Approve
4,CLM_000005,2023-04-20,2016-11-22,2023-04-04,2016,87272.268653,Electronics,ELEC_C,No_Start,Retail,Region_2,226.480249,334.526356,561.006604,0.734220,Reject,Reject


In [9]:
# Age at claim in years
df["Vehicle_Age_Years"] = (
    (df["Claim_Date"] - df["Vehicle_Registration_Date"])
    .dt.days
    .div(365.25)
)

# Days from failure to claim
df["Days_Failure_to_Claim"] = (
    (df["Claim_Date"] - df["Vehicle_Failure_Date"])
    .dt.days
)

# Sanity clip
df["Vehicle_Age_Years"] = df["Vehicle_Age_Years"].clip(lower=0)
df["Days_Failure_to_Claim"] = df["Days_Failure_to_Claim"].clip(lower=0)

df[["Vehicle_Age_Years", "Days_Failure_to_Claim"]].describe()


,Vehicle_Age_Years,Days_Failure_to_Claim
count,5000.000000,5000.000000
mean,4.195479,29.818000
std,2.264542,16.979474
min,0.273785,1.000000
25%,2.238877,15.000000
50%,4.193018,30.000000
75%,6.130048,44.000000
max,8.210815,59.000000


In [10]:
cols_order = [
    "Claim_ID",
    "Claim_Date",
    "Vehicle_Registration_Date",
    "Vehicle_Failure_Date",
    "Vehicle_MFD_Year",
    "Vehicle_Age_Years",
    "Days_Failure_to_Claim",
    "Mileage_km",
    "Part_Group",
    "Subpart_Code",
    "Failure_Mode",
    "Customer_Type",
    "Region",
    "Labor_Cost",
    "Material_Cost",
    "Total_Cost",
    "Burden_Ratio",
    "Final_Claim_Decision",
    "Final_DPR_Decision",
]

df = df[cols_order]

df.head()


,Claim_ID,Claim_Date,Vehicle_Registration_Date,Vehicle_Failure_Date,Vehicle_MFD_Year,Vehicle_Age_Years,Days_Failure_to_Claim,Mileage_km,Part_Group,Subpart_Code,Failure_Mode,Customer_Type,Region,Labor_Cost,Material_Cost,Total_Cost,Burden_Ratio,Final_Claim_Decision,Final_DPR_Decision
0,CLM_000001,2022-04-08,2020-06-04,2022-03-19,2020,1.842574,20,27512.065570,Chassis,CHS_A,No_Start,Retail,Region_2,154.603156,388.187881,542.791037,0.382099,Approve,Reject
1,CLM_000002,2024-04-27,2023-10-12,2024-04-14,2023,0.542094,13,4840.819371,Body,BODY_B,No_Start,Retail,Region_3,142.265157,68.022091,210.287249,0.791511,Approve,Reject
2,CLM_000003,2023-12-18,2018-09-16,2023-11-17,2018,5.253936,31,74398.919284,Electronics,ELEC_A,Leak,Retail,Region_2,159.957023,426.241409,586.198432,0.362170,Reject,Reject
3,CLM_000004,2023-04-26,2017-08-15,2023-03-17,2017,5.694730,40,58781.129706,Body,BODY_A,Electrical_Issue,Retail,Region_3,278.097472,168.986598,447.084071,0.621667,Approve,Approve
4,CLM_000005,2023-04-20,2016-11-22,2023-04-04,2016,6.406571,16,87272.268653,Electronics,ELEC_C,No_Start,Retail,Region_2,226.480249,334.526356,561.006604,0.734220,Reject,Reject


In [12]:
processed_dir = Path("../data/processed")
processed_dir.mkdir(parents=True, exist_ok=True)

processed_csv = processed_dir / "warranty_claims_processed.csv"

df.to_csv(processed_csv, index=False)

processed_csv

WindowsPath('../data/processed/warranty_claims_processed.csv')